## Importamos pandas y tweepy

In [1]:
import pandas
import tweepy

## Credenciales y autenticación del api

In [3]:


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

## Importar trends

In [3]:
woeidQuito = '375732'
trendsEc = api.trends_place(woeidQuito)

In [4]:
trendsEc

[{'trends': [{'name': '#HappyBirthdayToARMY',
    'url': 'http://twitter.com/search?q=%23HappyBirthdayToARMY',
    'promoted_content': None,
    'query': '%23HappyBirthdayToARMY',
    'tweet_volume': 366284},
   {'name': '#MorenoDebeResponder',
    'url': 'http://twitter.com/search?q=%23MorenoDebeResponder',
    'promoted_content': None,
    'query': '%23MorenoDebeResponder',
    'tweet_volume': None},
   {'name': 'IESS',
    'url': 'http://twitter.com/search?q=IESS',
    'promoted_content': None,
    'query': 'IESS',
    'tweet_volume': None},
   {'name': 'taehyun',
    'url': 'http://twitter.com/search?q=taehyun',
    'promoted_content': None,
    'query': 'taehyun',
    'tweet_volume': 186367},
   {'name': '#OurBorahaeDay',
    'url': 'http://twitter.com/search?q=%23OurBorahaeDay',
    'promoted_content': None,
    'query': '%23OurBorahaeDay',
    'tweet_volume': 222176},
   {'name': 'Koki',
    'url': 'http://twitter.com/search?q=Koki',
    'promoted_content': None,
    'query': 'K

## Devuelve los tweets en forma de DF

In [5]:
def getDataFrameTweets(tweets):
    index = 1
    
    # Creamos las columnas del df
    columnas = ( 'fecha', 'texto', 'retweets', 'favoritos', 'hashtags', 'menciones', 'localizacion','origen')
    
    # Creamos un df
    df = pandas.DataFrame(columns = columnas)
    
    for tweet in tweets:
        hashtags = tweet.entities['hashtags']
    
        listaHashtags = []

        for hashtag in hashtags:
            listaHashtags.append(hashtag['text'])

        user_mentions = tweet.entities['user_mentions']

        listaMenciones = []

        for users in user_mentions:
            listaMenciones.append(users['screen_name'])
        
        # Inserta cada una de las columnas
        df.loc[index] = tweet.created_at, tweet.text, tweet.retweet_count, tweet.favorite_count, listaHashtags, listaMenciones, tweet.user.location, tweet.source
        index = index + 1
    return df

## Devuelve los trends en forma de DF

In [6]:
def getDataFrameTrends(trends):
    index = 0
    
    columnas = ('trend', 'numTweetsLast24H')
    df = pandas.DataFrame(columns = columnas)
    
    for trend in trends[0]['trends']:
        df.loc[index] = trend['name'] , trend['tweet_volume']
        index = index + 1
    
    df.sort_values( by = ['numTweetsLast24H'], inplace = True)
    
    return df

## Generar DF de tweets a partir de los trendings y guardar

In [70]:
for trend in dfTrendsEc['trend']:
    tweets = api.search(trend, result_type = 'popular', count = 200)
    tweetsDf = getDataFrameTweets(tweets)
    nombreArchivo = trend + "Df.csv"
    tweetsDf.to_csv(nombreArchivo, sep = ';')

## Generar un solo DF con los tweets trending topic del momento 

In [16]:
# Creamos las columnas del df
columnas = ( 'fecha', 'texto', 'retweets', 'favoritos', 'hashtags', 'menciones', 'localizacion','origen', 'topic')
    
# Creamos un df
df = pandas.DataFrame(columns = columnas)
    
for trend in dfTrendsEc['trend']:
    tweets = api.search(trend, result_type = 'popular', count = 200)
    dfTemp = getDataFrameTweets(tweets)
    dfTemp = dfTemp.assign( topic = trend )  # Insertar nueva columna
    df = pandas.concat([df, dfTemp])

In [17]:
df

,fecha,texto,retweets,favoritos,hashtags,menciones,localizacion,origen,topic
1,2021-07-07 19:22:13,Tuliwaambia wamejazama CCM bungeni hakuna kitu...,201,1177,[],[],Exiled,Twitter for iPhone,Kitu
2,2021-07-08 05:57:28,"""There’s no reason for me to be President when...",67,367,[],[],|| R2P ||,Twitter for iPhone,Kitu
3,2021-07-08 09:05:40,Nadhani sasa kila mmoja amerudi kwenye factory...,77,642,[],[],Exiled,Twitter for iPhone,Kitu
4,2021-07-08 05:47:18,"""In other countries you have kickbacks of 10%....",17,97,[],[],"Nairobi, Kenya",Twitter Web App,Kitu
5,2021-07-08 02:40:00,Kitu gani kinajiri ulipo ? Tufahamishane @Vinc...,17,24,"[Jambokenya, Jambokenya, RadioNumberOne]","[VincentAteya, MelodySinzore1]",Communications Centre Nairobi,TweetDeck,Kitu
...,...,...,...,...,...,...,...,...,...
14,2021-07-04 18:35:29,Sou evangélico. Nossa mãe estudou em colégio d...,698,4648,[],[],,Twitter for Android,Católica
15,2021-07-08 17:15:56,#EnVideo 🎥🔍▶️ Iglesia católica dice que en #Ni...,15,43,"[EnVideo, Nicaragua]",[],Nicaragua,Twitter for Android,Católica
1,2021-07-08 16:38:59,"👨‍🏫🎙️ DT Fabián Bustos: ""Gabriel Marques está ...",42,198,[],[],"Guayaquil, Ecuador",Twitter Web App,barce
2,2021-07-08 16:12:02,"👨‍🏫🎙️ DT Fabián Bustos: ""Williams Riveros está...",40,176,[],[],"Guayaquil, Ecuador",Twitter Web App,barce


In [1]:
nombreArchivo = "TweetsTrendingEc-8Julio21.csv"
df.to_csv(nombreArchivo, sep = ';')

NameError: name 'df' is not defined

## Impresión de las tendencias en Ecuador

In [7]:
dfTrendsEc = getDataFrameTrends(trendsEc)
dfTrendsEc

,trend,numTweetsLast24H
17,Kitu,10248
14,Queso,16333
6,#FelizJueves,18922
42,Enserio,19002
40,Loja,22177
37,Unidad,26687
34,Ídolo,31116
22,Tokio,36397
26,tyler,43792
46,Juez,44396


### Impresión de los trends en el fichero

In [69]:
dfTrendsEc.to_csv('dfTendenciasEc8Jul21.csv', sep = ';')

In [40]:
trend = 'El Var'
nombreArchivo = trend + "Df.csv"
print(nombreArchivo)

El VarDf.csv


## Búsqueda de ejemplo

In [45]:
tweetsVar = api.search('El Var')

In [63]:
tweetsDf = getDataFrameTweets(tweetsVar)
tweetsDf

,fecha,texto,retweets,favoritos,hashtags,menciones,localizacion,origen
1,2021-06-26 21:19:28,"RT @alfunks: En un fútbol sin VAR, Italia esta...",6,0,[],[alfunks],"Cali, Colombia",Twitter for Android
2,2021-06-26 21:19:20,Con el VAR de la Eurocopa habríamos ganado la ...,0,0,[],[],Córdoba-Málaga-Madrid,Twitter for Android
3,2021-06-26 21:19:13,@aagulla_espn esto pasa despues de que el var ...,0,0,[],[aagulla_espn],,Twitter Web App
4,2021-06-26 21:19:09,"RT @alfunks: En un fútbol sin VAR, Italia esta...",6,0,[],[alfunks],,Twitter for Android
5,2021-06-26 21:19:07,@CopaAmerica @CONMEBOL @CBF_Futebol @FCFSelecc...,0,0,[],"[CopaAmerica, CONMEBOL, CBF_Futebol, FCFSelecc...","Puerto Rico, USA",Twitter for iPhone
6,2021-06-26 21:19:06,@asliparlakgibi Sınava ilk girişi değildi hoca...,0,0,[],[asliparlakgibi],"Antalya, Türkiye",Twitter for iPhone
7,2021-06-26 21:18:47,@calaco99 Con el. Var es otro deporte,0,1,[],[calaco99],mexico,Twitter for Android
8,2021-06-26 21:18:45,El var el!\n#EURO2020 #ITA #AUT #ITAUT #italy ...,0,0,"[EURO2020, ITA, AUT, ITAUT, italy, Austria]",[],,Twitter for Android
9,2021-06-26 21:18:45,RT @Scrat_Toston: Mi mujer me dijo “me haces f...,2,0,[],[Scrat_Toston],,Twitter for Android
10,2021-06-26 21:18:31,"RT @alfunks: En un fútbol sin VAR, Italia esta...",6,0,[],[alfunks],Guatemala,Twitter for iPhone


In [15]:
# Agregar user ->  entities -> hashtags | location | source |
for tweet in tweetsVar:
    print(tweet.entities['hashtags'])

[{'text': 'WAL', 'indices': [62, 66]}, {'text': 'DEN', 'indices': [71, 75]}]
[]
[]
[{'text': 'VARTND', 'indices': [0, 7]}]
[]
[{'text': 'VARTND', 'indices': [0, 7]}]
[]
[{'text': 'Deportes', 'indices': [0, 9]}, {'text': 'Noticias', 'indices': [105, 114]}]
[]
[{'text': 'ÖğrtErkenSeçimDiyor', 'indices': [93, 113]}]
[]
[]
[]
[]
[{'text': 'vartn', 'indices': [0, 6]}]


In [39]:
for tweet in tweetsVar:
    hashtags = tweet.entities['hashtags']
    
    listaHashtags = []
    
    for hashtag in hashtags:
        listaHashtags.append(hashtag['text'])
        
    user_mentions = tweet.entities['user_mentions']
    
    listaMenciones = []
    
    for users in user_mentions:
        listaMenciones.append(users['screen_name'])
        
    print(listaMenciones)
    

['DIRECTVSports']
['ANFPChile', 'LaRoja']
[]
[]
[]
['SaludEdomex']
[]
['PatronunF', 'el_patrone_bist']
['adilpsikoloji']
[]
[]
[]
[]
['CDeportivoLugo']
['pacogado49']


In [60]:
for tweet in tweetsVar:
    print(tweet.user.location)

Cali, Colombia
Córdoba-Málaga-Madrid


Puerto Rico, USA
Antalya, Türkiye
mexico


Guatemala
Barcelona, España
  Chilmyeonjo |  27ᵗʰ May 🍀 
yo soy de la cruz del sur 

España


In [43]:
print(tweetsVar)

[Status(_api=<tweepy.api.API object at 0x7efc900ad700>, _json={'created_at': 'Sat Jun 26 18:12:17 +0000 2021', 'id': 1408850618995941379, 'id_str': '1408850618995941379', 'text': "RT @DIRECTVSports: ⚽🇩🇰 ¡GOOOOOL DE DINAMARCA! 🇩🇰⚽A los 94' Martin Braithwaite pone el cuarto para la goleada danesa tras revisión del VAR.…", 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'DIRECTVSports', 'name': 'DIRECTV Sports', 'id': 133051204, 'id_str': '133051204', 'indices': [3, 17]}], 'urls': []}, 'metadata': {'iso_language_code': 'es', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1325972855310475264, 'id_str': '1325972855310475264', 'name': 'hatersismo', 'screen_name': 'hatersismo', 'location': 'Ate, P